In [ ]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [ ]:
from ASC_ML.networkbuilding import multiple_model_gen_v3 as multiple
from ASC_ML.networkbuilding import dataframe_extractor as de
from ASC_ML.networkbuilding import model_generation as model_gen
# from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# m = multiple.Multiple_Model_Gen(max_no_layers = 2, model_per_batch = 10)

In [ ]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [ ]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

In [ ]:
dataset_list[1].head()

In [ ]:
train_x = dataset_list[0].copy()

In [ ]:
train_x = train_x.fillna(0)

In [ ]:
train_x.head()

In [ ]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

In [ ]:
# a = train_x['LotShape']

In [ ]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [ ]:
# train_x.head()

In [ ]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [ ]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [ ]:
keys

In [ ]:
train_X = train_x.replace(keys).copy()

In [ ]:
train_X['FireplaceQu'].unique().compute()

In [ ]:
train_X.head()

In [ ]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [ ]:
print(cols_to_remove)

In [ ]:
train_X = train_X.drop(["Id"], axis = 1)

In [ ]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [ ]:
len(train_X.columns)

In [ ]:
len(dataset_list[1].columns)

In [ ]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [ ]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [ ]:
train_X_scaled
# train_Y_scaled

In [ ]:
train_Y_scaled

In [ ]:
print(train_X_scaled.shape)
print(train_Y_scaled.shape)

In [ ]:
import numpy as np
total_data = np.append(train_X_scaled, train_Y_scaled, axis = 1)

In [ ]:
print(total_data.shape)

In [ ]:
from sklearn.model_selection import train_test_split
(train, test) = train_test_split(total_data, test_size=0.25, random_state=42)

In [ ]:
trainX = train[:,:58]
trainY = train[:,-1]
testX = test[:,:58]
testY = test[:,-1]
print(trainX.shape)
print(trainY.shape)

In [ ]:
train_Y_scaled

In [ ]:
trainY = trainY.reshape((trainY.shape[0],1))
print(trainY.shape)

In [ ]:
testY = testY.reshape((testY.shape[0],1))

In [ ]:
start = time.time()
m = multiple.Multiple_Model_Gen_V3(trainX, trainY, testX, testY, 10, 128, input_shape = 58, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
m.get_model_confs()
m.get_best_models()
end = time.time()
print("Time Taken : ", (end-start))

In [ ]:
a = (int)t

## 2,3 Hidden Layers, 100 epochs, time taken = 147s (model saved)

In [ ]:
start = time.time()
m = multiple.Multiple_Model_Gen_V3(trainX, trainY, testX, testY, 10, 128, input_shape = 58, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
m.get_model_confs()
m.get_best_models()
end = time.time()
print("Time Taken : ", (end-start))

In [ ]:
m.evaluate_dict_list